# Layout

All UBC ubcpdk.components cells are conveniently combined into the ubcpdk.components module.

In [ ]:
import gdsfactory as gf

import ubcpdk

gf.CONF.display_type = "klayout"

## Fixed Component cells

Most `ubcpdk` components are imported from GDS files as fixed cells.

In [ ]:
c = ubcpdk.components.ebeam_crossing4()
c.plot()

In [ ]:
c = ubcpdk.components.ebeam_swg_edgecoupler()
c.plot()

In [ ]:
c = ubcpdk.components.ebeam_bdc_te1550()
c.plot()

In [ ]:
c = ubcpdk.components.ebeam_adiabatic_te1550()
c.plot()

In [ ]:
c = ubcpdk.components.ebeam_y_adiabatic()
c.plot()

In [ ]:
c = ubcpdk.components.ebeam_y_1550()
c.plot()

## Parametric Component PCells

You can also define cells adapted from gdsfactory generic pdk.

In [ ]:
c = ubcpdk.components.straight(length=2)
c.plot()

In [ ]:
c = ubcpdk.components.bend_euler(radius=5)
c.plot()

In [ ]:
c = ubcpdk.components.ring_with_crossing()
c.plot()

In [ ]:
c = ubcpdk.components.dbr()
c.plot()

In [ ]:
c = ubcpdk.components.spiral()
c.plot()

In [ ]:
c = ubcpdk.components.mzi_heater()
c.plot()

In [ ]:
c = ubcpdk.components.ring_single_heater()
c.plot()

## Components with grating couplers

To test your devices you can add grating couplers. Both for single fibers and for fiber arrays.

In [ ]:
splitter = ubcpdk.components.ebeam_y_1550(decorator=gf.port.auto_rename_ports)
mzi = gf.components.mzi(splitter=splitter)
mzi.plot()

In [ ]:
component_fiber_array = ubcpdk.components.add_fiber_array(component=mzi)
component_fiber_array.plot()

In [ ]:
c = ubcpdk.components.ring_single_heater()
c = ubcpdk.components.add_fiber_array_pads_rf(c)
c.plot()

In [ ]:
c = ubcpdk.components.mzi_heater()
c = ubcpdk.components.add_fiber_array_pads_rf(c, optical_routing_type=2)
c.plot()

## 3D rendering

In [ ]:
scene = c.to_3d()
scene.show()

## Die assembly

In [ ]:
from functools import partial
from pathlib import Path

import gdsfactory as gf

import ubcpdk
import ubcpdk.components as pdk
from ubcpdk import tech
from ubcpdk.tech import LAYER


size = (440, 470)
add_gc = ubcpdk.components.add_fiber_array
pack = partial(
    gf.pack, max_size=size, add_ports_prefix=False, add_ports_suffix=False, spacing=2
)

@gf.cell
def EBeam_JoaquinMatres_1() -> gf.Component:
    """Add DBR cavities."""
    e = [add_gc(ubcpdk.components.straight())]
    e += [add_gc(gf.components.mzi(delta_length=dl)) for dl in [9.32, 93.19]]
    e += [
        add_gc(gf.components.ring_single(radius=12, gap=gap, length_x=coupling_length))
        for gap in [0.2]
        for coupling_length in [2.5, 4.5, 6.5]
    ]

    e += [
        ubcpdk.components.dbr_cavity_te(w0=w0, dw=dw)
        for w0 in [0.5]
        for dw in [50e-3, 100e-3, 150e-3, 200e-3]
    ]
    e += [add_gc(ubcpdk.components.ring_with_crossing())]
    e += [
        add_gc(
            ubcpdk.components.ring_with_crossing(port_name="o2", with_component=False)
        )
    ]

    c = gf.Component()
    _ = c << pack(e)[0]
    _ = c << gf.components.rectangle(size=size, layer=LAYER.FLOORPLAN)
    return c

gf.clear_cache()
c = EBeam_JoaquinMatres_1()
c.show() # show in klayout
c.plot() # plot in notebook